In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet 

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import gensim

from google.cloud import bigquery

[nltk_data] Downloading package wordnet to /home/traffic/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/traffic/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/traffic/.local/lib/python3.8/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [2]:
# authenticate with BigQuery API
client = bigquery.Client.from_service_account_json('../../datascience-abovezero-58d98dcf7f65.json')
# Perform a query.
QUERY = ('SELECT * FROM `datascience-abovezero.ml_sandbox.chegg_influencers_comments`')
query_job = client.query(QUERY)  # API request
df =  query_job.result().to_dataframe() #transform to Pandas Dataframe

#df2 = pd.read_csv('analisis_comments_tiktok.csv')# Parameters tuning using Grid Search

# Bag Of Words

### Preprocessing 

In [3]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

comments = df.comment.tolist()
comments_prep = [preprocess(str(comment)) for comment in comments]
comments_dict = gensim.corpora.Dictionary(comments_prep)
bow_corpus = [comments_dict.doc2bow(doc) for doc in comments_prep]

In [4]:
'''
Checking dictionary created
'''
count = 0
for k, v in comments_dict.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 ayyyi
1 legend
2 call
3 colleg
4 univers
5 amaz
6 person
7 antedragz
8 chegg
9 fuck
10 gift


### Training the model

### Testing the model

# TFIDF 

### Preprocessing

In [5]:
docs_raw = df.comment.fillna('').tolist()
print(len(docs_raw))
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)

3427


/home/traffic/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:2029: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn(


### Training The Model 

In [11]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=4, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=4, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=4, random_state=0)

### Visualizing The Result 

In [16]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/home/traffic/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/traffic/.local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.090953  0.303184       1        1  22.978699
4     -0.352495 -0.047188       2        1  22.652388
1      0.175821 -0.260173       3        1  20.293166
0      0.181258  0.006986       4        1  17.879310
2     -0.095537 -0.002810       5        1  16.196438, topic_info=      Term       Freq      Total Category  logprob  loglift
25    love  68.000000  68.000000  Default  30.0000  30.0000
8    early  34.000000  34.000000  Default  29.0000  29.0000
1    chegg  38.000000  38.000000  Default  28.0000  28.0000
32     syd  27.000000  27.000000  Default  27.0000  27.0000
16    help  20.000000  20.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
25    love   0.214407  68.372968   Topic5  -6.3814  -3.9445
1    chegg   0.213832  38.032666   Topic5  -6.3840  -3.3606
21    like   0.213577  27.908427   Topic5  -6.3852  -3.0523
35  thanks   0.213553  19.802832   Topic5  -6.3853  -2.7093
38   video   0.213478  24.468883   Topic5  -6.3857  -2.9212

[215 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         4  0.893132    amazing
1         1  0.972848      chegg
2         1  0.389422      class
2         2  0.584133      class
3         1  0.121540    college
3         3  0.769752    college
3         5  0.121540    college
4         2  0.928783        day
5         2  0.846315        did
5         5  0.130202        did
6         4  0.695138       does
6         5  0.208541       does
7         3  0.260259        don
7         5  0.715713        don
8         4  0.986505      early
9         5  0.948012        eat
10        4  0.929332       free
11        1  0.434315       good
11        4  0.506701       good
12        1  0.917009        got
13        1  0.931434      great
14        2  0.977326      happy
15        2  0.652537    healthy
15        5  0.279659    healthy
16        4  0.975411       help
17        4  0.918984        hey
18        1  0.765489       just
18        5  0.214337       just
19        3  0.948751       know
20        3  0.267122       life
20        4  0.712326       life
21        1  0.214989       like
21        3  0.358315       like
21        4  0.429978       like
22        1  0.954630  literally
23        3  0.948743        lol
24        3  0.908592       look
25        2  0.994545       love
26        2  0.912210       make
27        3  0.908604       math
28        5  0.946064       need
29        5  0.945697        omg
30        2  0.941876     really
31        1  0.945221     school
32        5  0.980075        syd
33        5  0.945701     sydney
34        3  0.968305      thank
35        3  0.959459     thanks
36        1  0.928359        use
37        1  0.097599       used
37        2  0.097599       used
37        4  0.683193       used
38        1  0.980838      video
39        2  0.801117     videos
39        5  0.141374     videos
40        3  0.954360        yes, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 1, 3])

In [12]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/home/traffic/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/traffic/.local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.097591 -0.103921       1        1  31.065519
1      0.225117  0.138445       2        1  28.155732
2      0.085378 -0.163129       3        1  26.015309
0     -0.212903  0.128605       4        1  14.763441, topic_info=            Term        Freq       Total Category  logprob  loglift
67        cancer  443.000000  443.000000  Default  30.0000  30.0000
630        video  196.000000  196.000000  Default  29.0000  29.0000
515       school  295.000000  295.000000  Default  28.0000  28.0000
370         love  247.000000  247.000000  Default  27.0000  27.0000
92         colon  243.000000  243.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
346         like   35.273060  340.652731   Topic4  -4.5621  -0.3547
148      disease   22.165583   57.274251   Topic4  -5.0267   0.9637
140         didn   22.839465   81.988936   Topic4  -4.9967   0.6349
94   colonoscopy   23.320121  107.687931   Topic4  -4.9759   0.3831
139          did   21.816905  112.472857   Topic4  -5.0425   0.2730

[250 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         3  0.960698   advice
5         1  0.979321      age
5         2  0.016599      age
7         1  0.508058      ago
7         2  0.049167      ago
...     ...       ...      ...
675       4  0.282659    years
679       1  0.477660  youtube
679       3  0.057319  youtube
679       4  0.458553  youtube
681       4  0.954829     zach

[349 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1])